In [59]:
import base64
from datetime import datetime
import getpass
import json
import requests
from requests.auth import HTTPBasicAuth

This notebook details how to create and use EDL tokens to request data from NASA.

In [44]:
username = input('EDL Username')
password = getpass.getpass('EDL Password')

EDL Username aimeeb
EDL Password ··················


In [60]:
def find_or_get_token():
    urs_endoint = 'https://urs.earthdata.nasa.gov/api/users/token'
    # first try and retrieve an active token
    token_response = requests.get(f"{urs_endoint}s", auth=HTTPBasicAuth(username, password))
    error_message = "Attempting to get a existing tokens returned an error: {status_code} - {content}"
    if token_response.status_code != 200:
        return error_message.format(status_code=token_response.status_code, content=token_response.content)
    else:
        tokens = json.loads(token_response.content)
        for token in tokens:
            x_date = datetime.now()
            # I'm not sure if expired tokens are automatically deleted, need to ask
            # if not - should we revoke them?
            if token['expiration_date'] > x_date.strftime('%d/%m/%Y'):
                return token['access_token']

    # no active tokens were found      
    error_message = "Attempting to create a new token returned an error: {status_code} - {content}"    
    token_response = requests.post(urs_endoint, auth=HTTPBasicAuth(username, password))
    if token_response.status_code != 200:
        return error_message.format(status_code=token_response.status_code, content=token_response.content)
    return json.loads(token_response.content)['access_token']

In [95]:
# Simplified version of https://github.com/MAAP-Project/maap-py/blob/master/maap/Result.py#L89-L111
# This was tested and working for GEDI02_A.002 but not SENTINEL-1A_SLC
# From https://urs.earthdata.nasa.gov/users/aimeeb/user_tokens
# You can generate a bearer token for federated token access sharing. Not all EDL applications support Federated token access sharing.
# You may have up to 2 active tokens at a time.
# The token can then be passed into an application by using an Authorization: Bearer header
# The token will only authorize for applications that are EDL compliant and do not have unapproved EULAs
import os
import shutil
def get_data(url, destfile, destpath = '.'):
    api_header = {
        'Authorization': f"Bearer {find_or_get_token()}"
    }
    r = requests.get(
        url=url,
        headers=api_header,
        stream=True
    )

    if r.status_code != 200:
        raise ValueError('Bad search response for url {}: {}'.format(url, r.text))
    print(r.status_code)

    r.raw.decode_content = True

    with open(destpath + "/" + destfile, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

    return destpath + '/' + destfile

In [96]:
url = 'https://e4ftl01.cr.usgs.gov//GEDI_L1_L2/GEDI/GEDI02_A.002/2022.08.31/GEDI02_A_2022243234134_O21062_01_T10906_02_003_02_V002.h5'
destfile = url.split('/')[-1]
get_data(url, destfile)

200


'./GEDI02_A_2022243234134_O21062_01_T10906_02_003_02_V002.h5'